In [2]:
import json
from paperqa import Settings, ask
import os
from paperqa import Settings, Docs
from paperqa.agents.main import agent_query
from paperqa.agents.search import get_directory_index

from aviary.env import TaskDataset
from ldp.agent import SimpleAgent
from ldp.alg.callbacks import MeanMetricsCallback
from ldp.alg.runners import Evaluator, EvaluatorConfig
import sys

In [3]:
sys.path.append("src")

In [4]:
from MC_Grader import MultipleChoiceGrader, process_results, calculate_metrics_from_file
from config import OPENAI_API_KEY
from openai import OpenAI

## Load the Question 

In [5]:
with open("/Users/apple/Documents/GitLab_Projects/master_project/xx823/Reproduction/questions/formatted_questions_test/questions.json", "r") as f:
    questions = json.load(f)

In [6]:
print(questions)

[{'question': 'Approximately what percentage of topologically associated domains in the GM12878 blood cell line does DiffDomain classify as reorganized in the K562 cell line?', 'choices': ['(A) 41%', '(B) 51%', '(C) 31%', '(D) Insufficient information to answer the question', '(E) 11%', '(F) 21%'], 'correct_answer': 'C', 'unsure_option': 'D', 'sources': ['https://doi.org/10.1038/s41467-024-44782-6'], 'ideal': '31%', 'distractors': ['21%', '11%', '41%', '51%']}, {'question': 'At least how long do SynNotch-MCF10DCIS cells express BFP after contact with GFP+BMSC3 cells?', 'choices': ['(A) 0 h', '(B) 6 h', '(C) 24', '(D) 12 h', '(E) 48 h', '(F) 96 h', '(G) Insufficient information to answer the question', '(H) 72 h'], 'correct_answer': 'H', 'unsure_option': 'G', 'sources': ['https://doi.org/10.1073/pnas.2322688121'], 'ideal': '72 h', 'distractors': ['24', '48 h', '0 h', '12 h', '6 h', '96 h']}, {'question': 'DK015 and DK038 strains of Verticillium dahliae have in common approximately what 

In [7]:
def format_multiple_choice_question(question, choices):
    formatted = f"Question:Q: {question}; Options: "
    formatted += " ".join(choices)
    return formatted


In [8]:
formatted_q=[]

In [9]:
for q in questions:
    formatted_q.append(format_multiple_choice_question(q["question"], q["choices"]))

In [10]:
len(formatted_q)

49

In [11]:
print(formatted_q[:5])

['Question:Q: Approximately what percentage of topologically associated domains in the GM12878 blood cell line does DiffDomain classify as reorganized in the K562 cell line?; Options: (A) 41% (B) 51% (C) 31% (D) Insufficient information to answer the question (E) 11% (F) 21%', 'Question:Q: At least how long do SynNotch-MCF10DCIS cells express BFP after contact with GFP+BMSC3 cells?; Options: (A) 0 h (B) 6 h (C) 24 (D) 12 h (E) 48 h (F) 96 h (G) Insufficient information to answer the question (H) 72 h', 'Question:Q: DK015 and DK038 strains of Verticillium dahliae have in common approximately what percentage orthologous genes?; Options: (A) 97% (B) Insufficient information to answer the question (C) 95% (D) 98% (E) 96% (F) 94%', 'Question:Q: Expression of which of the following genes was significantly reduced in the cortical plate of olig2-/- mice compared to WT mice at E18.5?; Options: (A) none of the above (B) Insufficient information to answer the question (C) Aldh1l1 (D) Actin (E) MA

## Get the Paperqa Response

In [12]:
#build the index
# Create settings with explicit configuration
settings = Settings(
    paper_directory='/Users/apple/Documents/GitLab_Projects/master_project/xx823/papers',
    agent={"index": {
        "sync_with_paper_directory": True,
        "recurse_subdirectories": True
    }}
)

built_index = await get_directory_index(settings=settings)

# Print index information 
print(f"Using index: {settings.get_index_name()}")
index_files = await built_index.index_files
print(f"Number of indexed files: {len(index_files)}")
print("Indexed files:")
for file in index_files:
    print(f"- {file}")

Using index: pqa_index_73db35b7edb188828b5799b8ac0a3fac
Number of indexed files: 49
Indexed files:
- DiffDomain enables identification of structurally reorganized topologically associating domains.pdf
- Full-length tRNAs lacking a functional CCA tail are selectively sorted into the lumen of extracellular vesicles.pdf
- Type-I-interferon-responsive microglia shape cortical development and behavior.pdf
- liang-et-al-2024-a-conserved-molecular-logic-for-neurogenesis-to-gliogenesis-switch-in-the-cerebral-cortex.pdf
- goyette-et-al-2024-cancer-stromal-cell-interactions-in-breast-cancer-brain-metastases-induce-glycocalyx-mediated.pdf
- Light regulates widespread plant alternative polyadenylation 2 through the chloroplast.pdf
- High-speed imaging of giant unilamellar vesicle formation in cDICE.pdf
- elife-90425-v2.pdf
- sherman-et-al-2024-the-fatty-liver-disease-causing-protein-pnpla3-i148m-alters-lipid-droplet-golgi-dynamics.pdf
- Role of m6A RNA methylation in dosage compensation.pdf
- Func

In [13]:
from paperqa.settings import Settings, AgentSettings

In [14]:
def get_paperqa2_response(question_data):
    """
    This function should call PaperQA2 with the question and return the response.
    Adjust this function based on how you're currently calling PaperQA2.
    """
    # Load the settings
 
    settings = Settings(
    llm="gpt-4o-mini",
    llm_config={
        "model_list": [
            {
                "model_name": "gpt-4o-mini",
                "litellm_params": {
                    "model": "gpt-4o-mini",
                    "temperature": 0.1,
                    "max_tokens": 4096,
                },
            }
        ],
        "rate_limit": {
            "gpt-4o-mini": "30000 per 1 minute",
        },
    },
    summary_llm="gpt-4o-mini",
    summary_llm_config={
        "rate_limit": {
            "gpt-4o-mini": "30000 per 1 minute",
        },
    },
    agent=AgentSettings(
        agent_llm="gpt-4o-mini",
        agent_llm_config={
            "rate_limit": {
                "gpt-4o-mini": "30000 per 1 minute",
            },
        }
    ),
    embedding="text-embedding-3-small",
    temperature=0.5,  # Keep deterministic
    paper_directory="/Users/apple/Documents/GitLab_Projects/master_project/xx823/papers"  # Directory containing the paper with DOI: 10.1101/2024.01.31.578101
)

    # Format the question as needed for PaperQA2
    #format_multiple_choice_question(question_data['question'], question_data['choices'])

    import nest_asyncio
    nest_asyncio.apply()
    # Call PaperQA2 with the question
    response_pqa = ask(question_data, settings=settings)
    answer_dict=response_pqa.dict()

    return answer_dict['session']['answer']


## Create the required data structure

In [15]:
grader_input_data = []

In [16]:
formatted_q[0]

'Question:Q: Approximately what percentage of topologically associated domains in the GM12878 blood cell line does DiffDomain classify as reorganized in the K562 cell line?; Options: (A) 41% (B) 51% (C) 31% (D) Insufficient information to answer the question (E) 11% (F) 21%'

In [17]:
total_questions = len(formatted_q)
for i, q in enumerate(formatted_q[0]): # test for 49 questions
    current_question_num = i + 1
    print(f"Processing question {current_question_num}/{total_questions}: {q[:50]}...")
    
    # Get PaperQA2's response for this question
    response = get_paperqa2_response(q)
    
    # Log completion status
    print(f"Completed question {current_question_num}/{total_questions}, {total_questions - current_question_num} remaining")
    
    # Add the question and response to the grader input
    grader_input_data.append({
        "question":questions[i]["question"],
        "choices": questions[i]["choices"],
        "response": response,
        "correct_answer": questions[i]["correct_answer"],
        "unsure_option": questions[i]["unsure_option"]
    })

print(f"All {total_questions} questions processed successfully")


Processing question 1/49: Q...


[10:33:53] Starting paper search for 'Q'.

2025-03-04 10:33:53,836 - paperqa.agents.tools - INFO - Starting paper search for 'Q'.


           paper_search for query 'Q' and offset 0 returned 8 papers.

2025-03-04 10:33:53,909 - paperqa.agents.tools - INFO - paper_search for query 'Q' and offset 0 returned 8 papers.


...

           Status: Paper Count=8 | Relevant Papers=0 | Current Evidence=0 | Current Cost=$0.0001

2025-03-04 10:34:11,463 - paperqa.agents.tools - INFO - Status: Paper Count=8 | Relevant Papers=0 | Current Evidence=0 | Current Cost=$0.0001


[10:34:12] gather_evidence starting for question 'u'.

2025-03-04 10:34:12,488 - paperqa.agents.tools - INFO - gather_evidence starting for question 'u'.


KeyboardInterrupt: 

In [ ]:
with open("/Users/apple/Documents/GitLab_Projects/master_project/xx823/Reproduction/paperqa_response/grader_input_data_test_4omini.json", "w") as f:
    json.dump(grader_input_data, f, indent=2)

In [ ]:
#load the previous data if needed
with open("/Users/apple/Documents/GitLab_Projects/master_project/xx823/Reproduction/paperqa_response/grader_input_data_test_4omini.json", 'r') as f:
        grader_input_data = json.load(f)
    

In [ ]:
grader = MultipleChoiceGrader(
    # Add your OpenAI API key here if not set in environment
    openai_api_key=OPENAI_API_KEY,
    results_dir="grading_results_49MCQs_test_paperqa_4omini"
)


In [ ]:
# Use the fixed version instead of the original
process_results(grader, grader_input_data, save_prefix="paperqa2_evaluation")

2025-03-02 14:05:41,179 - MC_Grader - INFO - Processing question 1/49
2025-03-02 14:05:41,182 - MC_Grader - INFO - Processing question: Approximately what percentage of topologically ass...


HumanProxy (to GraderAgent):


    Question:
    Approximately what percentage of topologically associated domains in the GM12878 blood cell line does DiffDomain classify as reorganized in the K562 cell line?

    Options:
    (A) 41% (B) 51% (C) 31% (D) Insufficient information to answer the question (E) 11% (F) 21%

    PaperQA2 Response:
    DiffDomain identifies a significant proportion of topologically associated domains (TADs) that are reorganized when comparing the GM12878 blood cell line to the K562 cell line. Specifically, it reports that approximately 51.534% of TADs in GM12878 are classified as reorganized in the K562 cell line (hua2024diffdomainenablesidentification pages 3-4). This finding indicates that DiffDomain is effective in detecting structural changes in TADs between these two biological conditions.

Additionally, DiffDomain shows that a minimum of 43.137%, a median of 81.357%, and a maximum of 98.022% of the identified reorganized TADs in GM12878 are reorganized i

2025-03-02 14:05:44,227 - MC_Grader - INFO - Processing question 2/49
2025-03-02 14:05:44,229 - MC_Grader - INFO - Processing question: At least how long do SynNotch-MCF10DCIS cells expr...


HumanProxy (to GraderAgent):


    Question:
    At least how long do SynNotch-MCF10DCIS cells express BFP after contact with GFP+BMSC3 cells?

    Options:
    (A) 0 h (B) 6 h (C) 24 (D) 12 h (E) 48 h (F) 96 h (G) Insufficient information to answer the question (H) 72 h

    PaperQA2 Response:
    SynNotch-MCF10DCIS cells express BFP (Blue Fluorescent Protein) after contact with GFP+BMSC3 (Green Fluorescent Protein-positive Bone Marrow Stromal Cells) for at least 72 hours. The expression of BFP is observed to gradually decline after 24 hours but remains above baseline levels even at the 72-hour mark, indicating sustained expression during this period (goyette2024cancer–stromalcellinteractions pages 1-2). 

However, the exact duration of BFP expression beyond 72 hours is not clearly defined in the available literature. The study emphasizes the importance of maintaining both contact and no-contact populations in coculture models to assess gene expression changes, but it does not provide

2025-03-02 14:05:46,332 - MC_Grader - INFO - Processing question 3/49
2025-03-02 14:05:46,334 - MC_Grader - INFO - Processing question: DK015 and DK038 strains of Verticillium dahliae ha...


...

2025-03-02 14:07:09,989 - MC_Grader - INFO - Processing question 49/49
2025-03-02 14:07:09,990 - MC_Grader - INFO - Processing question: dimt-1 knockdown in C. elegans has been shown to c...


HumanProxy (to GraderAgent):


    Question:
    dimt-1 knockdown in C. elegans has been shown to cause upregulation of genes in which of the following pathways?

    Options:
    (A) FOXO signaling (B) WNT signaling (C) mTOR signaling (D) Insufficient information to answer the question (E) ribosome biogenesis

    PaperQA2 Response:
    The knockdown of dimt-1 in *Caenorhabditis elegans* has been shown to cause the upregulation of genes involved in several signaling pathways related to longevity regulation. Specifically, the pathways affected include TGF-β, WNT, and MAPK signaling pathways (rothi2024the18srrna pages 5-7; rothi2024the18srrna pages 19-20). Additionally, the dimt-1 knockdown influences the FoxO signaling pathway, which is crucial for lifespan regulation and stress resistance (rothi2024the18srrna pages 17-18). 

While the context does not explicitly mention mTOR signaling, it does indicate that the longevity effects of dimt-1 deficiency are linked to various biological pr

2025-03-02 14:07:11,878 - MC_Grader - INFO - Results saved to grading_results_49MCQs_test_paperqa_4omini



Results saved to grading_results_49MCQs_test_paperqa_4omini
- CSV: paperqa2_evaluation_extracted_answers.csv
- JSON: paperqa2_evaluation_extraction_results.json

You can now manually verify the results and calculate metrics.


'grading_results_49MCQs_test_paperqa_4omini/paperqa2_evaluation_extraction_results.json'

In [ ]:
metrics = calculate_metrics_from_file("/Users/apple/Documents/GitLab_Projects/master_project/xx823/Reproduction/grading_results_49MCQs_test_paperqa_4omini/paperqa2_evaluation_extraction_results.json")


Metrics:
Total questions: 49
Correct answers: 34 (69.39%)
Incorrect answers: 8 (16.33%)
Unsure answers: 7 (14.29%)
Precision: 80.95%
Accuracy: 69.39%
Metrics saved to /Users/apple/Documents/GitLab_Projects/master_project/xx823/Reproduction/grading_results_49MCQs_test_paperqa_4omini/metrics.csv
